Interaction data is the most basic indication of users’ preferences and interests but is usually extremely sparse and can be noisy at times.

To address this issue, we can integrate side information such as features of items, profiles of users, and even in which context that the interaction occurred into the recommendation model.

To demonstrate this type of recommendation models, we introduce another task on click-through rate (CTR) for online advertisement recommendations.

$\text{CTR} = \frac{\#\text{Clicks}} {\#\text{Impressions}} \times 100 \%$

Click-through rate is an important signal that indicates the effectiveness of prediction algorithms.

In [ ]:
!pip install mxnet-cu101==1.7.0

In [ ]:
!pip install d2l==0.17.5

In [3]:
import os
from collections import defaultdict
from mxnet import gluon, np
from d2l import mxnet as d2l

### An Online Advertising Dataset

It consists of 34 fields, with the first column representing the target variable that indicates if an ad was clicked (1) or not (0).

In [4]:
d2l.DATA_HUB['ctr'] = (d2l.DATA_URL + 'ctr.zip',
                       'e18327c48c8e8e5c23da714dd614e390d369843f')

data_dir = d2l.download_extract('ctr')

### Dataset Wrapper

In [5]:
class CTRDataset(gluon.data.Dataset):
    def __init__(self, data_path, feat_mapper=None, defaults=None,
                 min_threshold=4, num_feat=34):
        self.NUM_FEATS, self.count, self.data = num_feat, 0, {}
        feat_cnts = defaultdict(lambda: defaultdict(int))
        self.feat_mapper, self.defaults = feat_mapper, defaults
        self.field_dims = np.zeros(self.NUM_FEATS, dtype=np.int64)
        with open(data_path) as f:
            for line in f:
                instance = {}
                values = line.rstrip('\n').split('\t')
                if len(values) != self.NUM_FEATS + 1:
                    continue
                label = np.float32([0, 0])
                label[int(values[0])] = 1
                instance['y'] = [np.float32(values[0])]
                for i in range(1, self.NUM_FEATS + 1):
                    feat_cnts[i][values[i]] += 1
                    instance.setdefault('x', []).append(values[i])
                self.data[self.count] = instance
                self.count = self.count + 1
        if self.feat_mapper is None and self.defaults is None:
            feat_mapper = {i: {feat for feat, c in cnt.items() if c >=
                               min_threshold} for i, cnt in feat_cnts.items()}
            self.feat_mapper = {i: {feat_v: idx for idx, feat_v in enumerate(feat_values)}
                                for i, feat_values in feat_mapper.items()}
            self.defaults = {i: len(feat_values) for i, feat_values in feat_mapper.items()}
        for i, fm in self.feat_mapper.items():
            self.field_dims[i - 1] = len(fm) + 1
        self.offsets = np.array((0, *np.cumsum(self.field_dims).asnumpy()
                                 [:-1]))

    def __len__(self):
        return self.count

    def __getitem__(self, idx):
        feat = np.array([self.feat_mapper[i + 1].get(v, self.defaults[i + 1])
                         for i, v in enumerate(self.data[idx]['x'])])
        return feat + self.offsets, self.data[idx]['y']

In [6]:
train_data = CTRDataset(os.path.join(data_dir, 'train.csv'))
train_data[0]

(array([ 143.,  144.,  227.,  254.,  957., 1250., 1471., 1566., 1624.,
        1990., 2008., 2061., 2086., 2304., 2305., 2360., 2745., 2746.,
        2747., 2748., 2892., 2988., 3165., 3190., 3194., 3195., 3549.,
        3652., 3687., 3704., 3722., 3753., 3779., 3799.]), [1.0])